<a href="https://colab.research.google.com/github/crystalloide/Notebooks-SQL-serie-1/blob/main/ex08_Jointure_entre_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ex08 - Jointure entre tables

Lorsque nous concevons un système de base de données complet en utilisant de bons principes de conception tels que la normalisation, différents aspects des informations doivent être séparés en tables normalisées. 

Dans un tel cas, nous avons souvent besoin de ***jointures*** pour récupérer des données de plusieurs tables dans une seule requête SELECT. 

Deux tables peuvent être jointes par un seul opérateur de jointure, mais le résultat peut être joint à nouveau avec d'autres tables. 

Il doit exister une colonne identique ou similaire entre les tables jointes.

Pour connecter des tables dans une requête, nous utilisons une instruction ***JOIN ... ON***. 

Il existe différents types de jointures SQLite :
- INNER JOIN (ou parfois appelé jointure simple)
- LEFT OUTER JOIN (ou parfois appelé LEFT JOIN)
- JOINTURE CROISÉE

La jointure de type RIGHT OUTER JOIN et FULL OUTER JOIN ne sont pas prises en charge dans SQLite.

In [1]:
%load_ext sql

In [2]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connection à la database demo.db3

In [3]:
%sql sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3

'Connected: @/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3'

Si vous ne vous souvenez pas des tables présentes dans la database de démonstration, vous pouvez toujours utiliser la commande suivante pour les retrouver.

In [4]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. INNER JOIN

L'INNER JOIN nous permet de fusionner deux tables ensemble. Mais si nous allons fusionner des tables, nous devons définir un point commun entre les deux afin que les enregistrements des deux tables s'alignent. Nous devons définir un ou plusieurs champs qu'ils ont en commun et les rejoindre.

#### 2.1 Vérification des colonnes en commun entre deux tables

Ici, nous prenons les tableaux de ***rch*** et ***sub*** comme exemple. 
Il y a trois colonnes communes de RCH / SUB, YR et MO.


In [5]:
%sql SELECT * From rch LIMIT 3

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,EVAPcms,TLOSScms,SED_INtons,SED_OUTtons,SEDCONCmg_kg,ORGN_INkg,ORGN_OUTkg,ORGP_INkg,ORGP_OUTkg,NO3_INkg,NO3_OUTkg,NH4_INkg,NH4_OUTkg,NO2_INkg,NO2_OUTkg,MINP_INkg,MINP_OUTkg,CHLA_INkg,CHLA_OUTkg,CBOD_INkg,CBOD_OUTkg,DISOX_INkg,DISOX_OUTkg,SOLPST_INmg,SOLPST_OUTmg,SORPST_INmg,SORPST_OUTmg,REACTPSTmg,VOLPSTmg,SETTLPSTmg,RESUSP_PSTmg,DIFFUSEPSTmg,REACBEDPSTmg,BURYPSTmg,BED_PSTmg,BACTP_OUTct,BACTLP_OUTct,CMETAL_1kg,CMETAL_2kg,CMETAL_3kg,TOT_Nkg,TOT_Pkg,NO3ConcMg_l,WTMPdegc
1,1981,1,146.34376525878906,146.2524871826172,0.09128088504076004,0.0,2.3320461650655488e-07,61619.46484375,155.3719024658203,0.016086287796497345,0.0,0.04825887829065323,0.0,362.0486755371094,361.8135070800781,203.620849609375,421.18377685546875,0.0,23.0184326171875,0.016107244417071342,0.0,1.1839052307949238e-11,0.0,0.0,0.0,5627225.0,5623486.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,806.0157470703125,0.0,0.0,0.0
2,1981,1,96.22569274902344,96.18285369873047,0.042821284383535385,0.0,1.6426764659627224e-07,0.0,0.0,0.01363153662532568,0.0,0.04089462012052536,0.0,315.60052490234375,315.4579772949219,0.0,127.00502014160156,0.0,0.0,0.013656049966812134,0.0,4.136972177163251e-16,0.0,0.0,0.0,3757606.5,3698301.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,442.4630126953125,0.0,0.0,0.0
3,1981,1,11.952718734741211,11.861368179321289,0.09135182946920395,0.0,2.0325823868461157e-07,2.0325823868461157e-07,6.595060941805286e-09,0.01146628800779581,0.0,0.03439886122941971,0.009118038229644299,48.2963752746582,47.93150329589844,0.0,62.467620849609375,0.0,0.0,0.011485105380415916,0.0,5.941028783295818e-14,0.0,0.0,0.0,360979.90625,456115.90625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.39912414550781,0.009118038229644299,0.0,0.0


In [6]:
%sql SELECT * From sub LIMIT 3

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


SUB,YR,MO,PRECIPmm,SNOMELTmm,PETmm,ETmm,SWmm,PERCmm,SURQmm,GW_Qmm,WYLDmm,SYLDt_ha,ORGNkg_ha,ORGPkg_ha,NSURQkg_ha,SOLPkg_ha,SEDPkg_ha,LAT_Q_mm,LATNO3kg_h,GWNO3kg_ha,CHOLAmic_L,CBODU_mg_L,DOXQ_mg_L,TNO3kg_ha,QTILEmm,TVAPkg_ha
1,1981,1,35.60198974609375,0.0,3.7207436561584473,0.24964235723018646,10.79859447479248,0.0,2.3795111701474525e-05,0.05372961238026619,0.2988955080509186,3.4497936191729694e-13,2.37964208338326e-08,2.37964208338326e-08,2.37964208338326e-08,2.37964208338326e-08,4.75928416676652e-08,0.0078118653036653996,0.0,1.5622873661413905e-07,0.0,0.0,0.0,0.0,0.0,0.0
2,1981,1,108.60607147216797,0.0,3.4504077434539795,0.45720556378364563,56.32500457763672,0.0,1.7237898646271788e-05,28.572050094604492,32.59342956542969,2.077378118700915e-13,1.7238855676282583e-08,1.7238855676282583e-08,1.7238855676282583e-08,1.7238855676282583e-08,3.4477711352565166e-08,0.03393149375915527,1.6220961697399616e-05,0.00038287980714812875,0.0,0.0,0.0,0.0,0.0,0.0
3,1981,1,149.30836486816406,0.0,10.566324234008789,6.027106285095215,71.0020980834961,0.0,2.4236529498011805e-05,3.9627554416656494,5.274456977844238,3.348758173971761e-13,1.8891142872234923e-08,1.8891142872234923e-08,1.8891142872234923e-08,1.8891142872234923e-08,3.7782285744469846e-08,0.00791492685675621,3.7224147035885835e-06,7.582882244605571e-05,0.0,0.0,0.0,0.0,0.0,0.0


#### 2.2 Faire une jointure de type "inner join"

La syntaxe pour une INNER JOIN dans SQLite est la suivante :
>SELECT columns<br>
>FROM table1<br> 
>INNER JOIN table2<br>
>ON table1.column = table2.column;

Fait la jointure entre RCH/SUB, YR and MO. 

**Remarque** : Lors du SELECT (sélection) des colonnes communes, vous devrez explicitement attribuer un nom de table. 

Si les noms de colonne ou de table sont trop longs, il est possible d'utiliser des alias pour donner des noms plus courts.


In [7]:
%%sql sqlite://
SELECT RCH, rch.YR, rch.MO, FLOW_INcms, FLOW_OUTcms, PRECIPmm, PETmm
FROM rch INNER JOIN sub
ON rch.RCH = sub.SUB and rch.YR=sub.YR and rch.MO=sub.MO
LIMIT 5

Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
1,1981,1,146.34376525878906,146.2524871826172,35.60198974609375,3.7207436561584473
2,1981,1,96.22569274902344,96.18285369873047,108.60607147216797,3.4504077434539795
3,1981,1,11.952718734741211,11.861368179321289,149.30836486816406,10.566324234008789
4,1981,1,49.48649215698242,49.40651321411133,108.60604858398438,10.674993515014648
5,1981,1,274.0668029785156,272.10601806640625,201.311279296875,27.179243087768555


### 3. LEFT JOIN

Semblable à la clause INNER JOIN, la clause LEFT JOIN est une clause facultative de l'instruction SELECT. 

La clause LEFT JOIN permet d'interroger les données de plusieurs tables corrélées. 

Ce type de jointure renvoie toutes les lignes de la table GAUCHE spécifiée dans la condition ON et uniquement les lignes de l'autre table où les champs joints sont égaux (la condition de jointure est remplie).

La syntaxe de SQLite LEFT OUTER JOIN est la suivante:
>SELECT columns<br>
>FROM table1<br>
>LEFT [OUTER] JOIN table2<br>
>ON table1.column = table2.column;

In [8]:
%%sql sqlite://
SELECT RCH, rch.YR, rch.MO, FLOW_INcms, FLOW_OUTcms, PRECIPmm, PETmm
FROM rch LEFT JOIN sub
ON rch.RCH = sub.SUB and rch.YR=sub.YR and rch.MO=sub.MO
LIMIT 5

Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
1,1981,1,146.34376525878906,146.2524871826172,35.60198974609375,3.7207436561584473
2,1981,1,96.22569274902344,96.18285369873047,108.60607147216797,3.4504077434539795
3,1981,1,11.952718734741211,11.861368179321289,149.30836486816406,10.566324234008789
4,1981,1,49.48649215698242,49.40651321411133,108.60604858398438,10.674993515014648
5,1981,1,274.0668029785156,272.10601806640625,201.311279296875,27.179243087768555


### 4. CROSS JOIN

Another type of join is called a SQLite CROSS JOIN. This type of join returns a combined result set with every row from the first table matched with every row from the second table. This is also called a Cartesian Product.

Une autre sorte de jointure est appelée dans SQLite une jointure de type CROSS JOIN. 

Ce type de jointure renvoie un jeu de résultats combiné avec chaque ligne de la première table correspondant à chaque ligne de la deuxième table. 

Ceci est également appelé un produit cartésien.

La syntaxe dans SQLite de CROSS JOIN est:

>SELECT columns<br>
>FROM table1<br>
>CROSS JOIN table2;

In [9]:
%%sql sqlite://
SELECT RCH, rch.YR, rch.MO, FLOW_INcms, FLOW_OUTcms, PRECIPmm, PETmm
FROM rch
CROSS JOIN sub
LIMIT 5

Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
1,1981,1,146.34376525878906,146.2524871826172,35.60198974609375,3.7207436561584473
1,1981,1,146.34376525878906,146.2524871826172,108.60607147216797,3.4504077434539795
1,1981,1,146.34376525878906,146.2524871826172,149.30836486816406,10.566324234008789
1,1981,1,146.34376525878906,146.2524871826172,108.60604858398438,10.674993515014648
1,1981,1,146.34376525878906,146.2524871826172,201.311279296875,27.179243087768555


### 5. Requêtage de plusieurs tables à l'aide de JOIN

Les bases de données relationnelles peuvent être assez complexes en termes de relations entre les tables. Parfois, nous devons récupérer des informations provenant de plus de deux tables.

Nous pouvons utiliser la syntaxe suivante pour joindre plusieurs tables:
>SELECT columns<br>
>FROM table1<br>
>INNER JOIN table2 ON table1.column = table2.column<br>
>INNER JOIN table3 ON table1.column = table3.column<br>
>...<br>
>INNER JOIN tablen ON table1.column = tablen.column;

Dans le langage SQL proprement dit, il n'y a pas de limite au nombre maximum de tables que vous pouvez joindre.

Cependant, la plupart des SGBD ont leurs propres limites. Ce sera donc à vérifier en pratique en fonction du SGBD utilisé.

De plus, la requête ralentira considérablement lors de la jonction d'un trop grand nombre de tables (par exemple, 4 tables ou plus). 
La performance dans un contexte réel en production s'en ressentira.

### Conclusion

Dans ce Notebook, nous avons pratiqué les trois principaux types de jointures dans SQLite: à savoir les jointures INNER, LEFT et CROSS. 

Les jointures nous permettent de récupérer des données dispersées sur plusieurs tables et de les assembler en quelque chose de plus significatif et descriptif. 

Nous pouvons prendre deux tables ou plus et les réunir dans une table plus grande qui a plus de contexte. 

De plus, l'utilisation d'alias nous permet de renommer les noms de colonnes ou de tables à la volée.